Victor CHAU & Tahar AMAIRI
# IFT 870 - Forage de données
## TP2 : Prétraitement et représentation de données

### Sommaire

* Question 1 & Question 2
* Question 3
* Question 4
* Question 5
* Question 6
* Question 7
* Question 8
* Question 9

## Importation des données

In [459]:
# import libs
from matplotlib import pyplot as plt
from IPython.display import display
import seaborn as sns
import pandas as pd
import numpy as np
import datetime
import re

# read CSVs
dfProd = pd.read_csv('./product2.csv', sep=';', encoding='latin1')
dfPack = pd.read_csv('./package2.csv', sep=';', encoding='latin1')

# print
print("Product table:")
display(dfProd.head())
print("Package table:")
display(dfPack.head())

Product table:


,PRODUCTID,PRODUCTNDC,PRODUCTTYPENAME,PROPRIETARYNAME,PROPRIETARYNAMESUFFIX,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,STARTMARKETINGDATE,ENDMARKETINGDATE,MARKETINGCATEGORYNAME,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,DEASCHEDULE,NDC_EXCLUDE_FLAG,LISTING_RECORD_CERTIFIED_THROUGH
0,NaN,0002-0800,HUMAN OTC DRUG,Sterile Diluent,NaN,diluent,"INJECTION, SOLUTION",SUBCUTANEOUS,19870710,NaN,NDA,NDA018781,10,WATER,1,mL/mL,NaN,NaN,N,20201231.0
1,NaN,0002-1200,HUMAN PRESCRIPTION DRUG,Amyvid,NaN,Florbetapir F 18,"INJECTION, SOLUTION",INTRAVENOUS,20120601,NaN,NDA,NDA202008,10,FLORBETAPIR F-18,51,mCi/mL,"Radioactive Diagnostic Agent [EPC],Positron Em...",NaN,N,20211231.0
2,NaN,0002-1433,HUMAN PRESCRIPTION DRUG,Trulicity,NaN,Dulaglutide,"INJECTION, SOLUTION",SUBCUTANEOUS,20140918,NaN,BLA,BLA125469,10,DULAGLUTIDE,.75,mg/.5mL,"GLP-1 Receptor Agonist [EPC],Glucagon-Like Pep...",NaN,N,20201231.0
3,NaN,0002-1434,HUMAN PRESCRIPTION DRUG,Trulicity,NaN,Dulaglutide,"INJECTION, SOLUTION",SUBCUTANEOUS,20140918,NaN,BLA,BLA125469,10,DULAGLUTIDE,1.5,mg/.5mL,"GLP-1 Receptor Agonist [EPC],Glucagon-Like Pep...",NaN,N,20201231.0
4,NaN,0002-1436,HUMAN PRESCRIPTION DRUG,EMGALITY,NaN,galcanezumab,"INJECTION, SOLUTION",SUBCUTANEOUS,20180927,NaN,BLA,BLA761063,10,GALCANEZUMAB,120,mg/mL,NaN,NaN,N,20201231.0


Package table:


,PRODUCTID,PRODUCTNDC,NDCPACKAGECODE,PACKAGEDESCRIPTION,STARTMARKETINGDATE,ENDMARKETINGDATE,NDC_EXCLUDE_FLAG,SAMPLE_PACKAGE
0,0002-0800_94c48759-29bb-402d-afff-9a713be11f0e,0002-0800,0002-0800-01,1 VIAL in 1 CARTON (0002-0800-01) > 10 mL in ...,19870710,NaN,N,N
1,0002-1200_35551a38-7a8d-43b8-8abd-f6cb7549e932,0002-1200,0002-1200-30,"1 VIAL, MULTI-DOSE in 1 CAN (0002-1200-30) > ...",20120601,NaN,N,N
2,0002-1200_35551a38-7a8d-43b8-8abd-f6cb7549e932,0002-1200,0002-1200-50,"1 VIAL, MULTI-DOSE in 1 CAN (0002-1200-50) > ...",20120601,NaN,N,N
3,0002-1433_42a80046-fd68-4b80-819c-a443b7816edb,0002-1433,0002-1433-61,2 SYRINGE in 1 CARTON (0002-1433-61) > .5 mL ...,20141107,NaN,N,Y
4,0002-1433_42a80046-fd68-4b80-819c-a443b7816edb,0002-1433,0002-1433-80,4 SYRINGE in 1 CARTON (0002-1433-80) > .5 mL ...,20141107,NaN,N,N


### Question 1 & Question 2

Pour des raisons de simplicité, nous allons répondre à ces deux questions simultanément : en effet, la majorité des incohérences sont causées par un non respect de certaines règles. Il serait donc redondant de traiter les deux questions séparément lorsqu'en réalité elles sont intrinsèquement liées.

In [460]:
# a simple function showing some information about the passed df
def describeDF(df):
    end = '\033[0m'
    underline = '\033[4m'

    print(underline + "Data frame shape:" + end)
    print(df.shape)
    
    print(underline + "Columns types:" + end)
    print(df.dtypes)

    print(underline + "Percentage of missing values for each columns:" + end)
    print(df.isnull().sum() * 100 / len(df))

In [461]:
# product table
describeDF(dfProd)

Data frame shape:
(93238, 20)
Columns types:
PRODUCTID                            object
PRODUCTNDC                           object
PRODUCTTYPENAME                      object
PROPRIETARYNAME                      object
PROPRIETARYNAMESUFFIX                object
NONPROPRIETARYNAME                   object
DOSAGEFORMNAME                       object
ROUTENAME                            object
STARTMARKETINGDATE                    int64
ENDMARKETINGDATE                    float64
MARKETINGCATEGORYNAME                object
APPLICATIONNUMBER                    object
LABELERNAME                          object
SUBSTANCENAME                        object
ACTIVE_NUMERATOR_STRENGTH            object
ACTIVE_INGRED_UNIT                   object
PHARM_CLASSES                        object
DEASCHEDULE                          object
NDC_EXCLUDE_FLAG                     object
LISTING_RECORD_CERTIFIED_THROUGH    float64
dtype: object
Percentage of missing values for each columns:
PRODUCTID     

Commençons tout d'abord par la table produit, elle contient 93238 lignes et 20 attributs :

- ``PRODUCTID`` : Cette colonne est la concaténation du code NDC de la colonne ``PRODUCTNDC`` et d'une donnée inconnue nommée ``SPL`` séparée par un tiret. Elle ne doit pas contenir de valeur NULL et doit être de type string. Or, elle contient 1.67 % de NULL. Cependant, elle n'est d'aucune utilité à part pour join les deux tables.

- ``PRODUCTNDC`` : Cette colonne est la concaténation d'un code NDC et du code de l'étiqueteur séparée par un tiret. Elle ne doit pas contenir de valeur NULL et doit être de type string. On pourra vérifier que pour chaque ligne on a bien la cohérence inter-attribut avec la colonne ``PRODUCTID`` (au niveau du code NDC).

- ``PRODUCTTYPENAME`` : Cette colonne indique le type de produit. Elle ne doit pas contenir de valeur NULL et doit être de type string. On pourra étudier les valeurs qu'elle contient pour voir s'ils sont cohérents par rapport à une description d'un médicament.

- ``PROPRIETARYNAME`` : Cette colonne contient le nom commercial du produit. Elle ne doit pas contenir de valeur NULL et doit être de type string. Or, elle contient un faible niveau de valeur NULL (0.006435 %).

- ``PROPRIETARYNAMESUFFIX`` : Cette colonne contient le suffixe du nom commercial du produit. Elle peut contenir des valeurs NULL et doit être de type string. On remarque un niveau élevé de valeur NULL (89 %), ce qui la rend d'une faible utilité car elle est vraiment optionnelle. De plus, on ne peut vérifier le format de chaque entrée car il n'existe aucun standard et chaque fabricant utilise sa propre convention.

- ``NONPROPRIETARYNAME`` : Cette colonne contient une liste de string décrivant les noms génériques du produit et ne peut pas contenir des valeurs NULL. Or, elle contient un faible niveau de valeur NULL (0.004290 %).

- ``DOSAGEFORMNAME`` : Cette colonne décrit la forme du médicament. Elle ne doit pas contenir de valeur NULL et doit être de type string. On pourra vérifier la cohérence des valeurs avec la liste de code fourni par la FDA (voir le fichier ``DosageForms.txt``).

- ``ROUTENAME`` : Cette colonne contient une liste de string décrivant les moyens d'administration du produit et ne peut pas contenir des valeurs NULL. Or, elle contient un faible niveau de valeur NULL (2.072117 %). On pourra vérifier la cohérence des valeurs avec la liste de code fourni par la FDA (voir le fichier ``RouteNames.txt``).

- ``STARTMARKETINGDATE`` : Cette colonne contient la date de la mise en marché du produit. Elle ne doit pas contenir de valeur NULL et doit être de type string. Or, on remarque qu'elle est de type ``int``. Il sera donc nécessaire de la transformer en date et de vérifier si elles sont cohérentes.

- ``ENDMARKETINGDATE`` : Cette colonne contient la date de la fin de la commercialisation du produit. Elle peut contenir des valeur NULL et doit être de type string. Or, on remarque qu'elle est de type ``float``. Il sera donc nécessaire de la transformer en date et de vérifier si elles sont cohérentes. Par ailleurs, on remarque qu'elle contient énormément de valeur NULL (95 %), ce qui signifie que la majorité des produits sont toujours en train d'être fabriqués et commercialisés, ce qui réduit l'importance de cette colonne. 

- ``MARKETINGCATEGORYNAME`` : Cette colonne contient la catégorie du produit. Elle ne doit pas contenir de valeur NULL et doit être de type string. On pourra vérifier la cohérence des valeurs avec la liste de code fourni par la FDA (voir le fichier ``MarketingCatNames.txt``).

- ``APPLICATIONNUMBER`` : Cette colonne contient le numéro d'application du produit. Elle doit être de type string et une valeur NULL signifie que le produit n'est pas approuvé par la FDA. Il contient un suffixe qui est identique à la valeur dans la colonne ``MARKETINGCATEGORYNAME`` et cela est à vérifier. 

- ``LABELERNAME`` : Cette colonne contient le nom de l'étiqueteur. Elle ne doit pas contenir de valeur NULL et doit être de type string.

- ``SUBSTANCENAME``, ``ACTIVE_NUMERATOR_STRENGTH``, ``ACTIVE_INGRED_UNIT`` : Ces 3 attributs sont liés et décrivent la composition chimique du produit. Ils ne doivent pas contenir de valeur NULL et doivent être de type string. Or, on remarque que chacune de ces colonnes possède le même pourcentage de NULL ce qui signifie que si une ligne possède une valeur NULL dans l'une des colonnes alors cela doit être le cas pour les deux autres. On pourra vérifier si elles contiennent le même nombre d'éléments car chaque attribut contient des listes. Par ailleurs, on peut aussi vérifier si la colonne ``ACTIVE_NUMERATOR_STRENGTH`` contient des digits.

- ``PHARM_CLASSES`` : Cette colonne décrit via une liste de string les classes pharmaceutiques des substances de la colonne ``SUBSTANCENAME``. Elle ne doit pas contenir de valeur NULL. Or, on remarque qu'elle contient 54.681568 % de valeur NULL.

- ``DEASCHEDULE`` : Cette colonne contient des strings avec pour valeur : CI, CII, CIII, CIV, et CV. Elle ne doit pas contenir de valeur NULL. Or, on remarque qu'elle a un niveau élevé de valeur NULL (95.256226 %). Elle indique le pouvoir de dépendance du produit, on peut donc imaginer si un produit a une valeur NULL, alors celui-ci n'a aucun pouvoir de dépendance.  

- ``NDC_EXCLUDE_FLAG`` : Cette colonne indique si le produit est exclu des fichiers du NDC pour X raisons. Dans notre cas, le fichier chargé ne contient que des produits acceptés. Pour cela, normalement l'unique valeur contenue dans cette colonne est ``N``, ce qu'il faudra vérifier.

- ``LISTING_RECORD_CERTIFIED_THROUGH`` : Cette colonne contient la date d’expiration de l'inscription du produit si pas mis à jour par l'entreprise. Comme pour les autres colonnes contenant des dates, il faut la transformer en date et vérifier sa cohérence. Finalement, elle ne doit pas contenir de valeur NULL, or, elle en contient très peu (4.638667 %).

In [462]:
# package table
describeDF(dfPack)

Data frame shape:
(173887, 8)
Columns types:
PRODUCTID              object
PRODUCTNDC             object
NDCPACKAGECODE         object
PACKAGEDESCRIPTION     object
STARTMARKETINGDATE      int64
ENDMARKETINGDATE      float64
NDC_EXCLUDE_FLAG       object
SAMPLE_PACKAGE         object
dtype: object
Percentage of missing values for each columns:
PRODUCTID              0.000000
PRODUCTNDC             0.862629
NDCPACKAGECODE         1.349152
PACKAGEDESCRIPTION     0.000000
STARTMARKETINGDATE     0.000000
ENDMARKETINGDATE      96.287244
NDC_EXCLUDE_FLAG       0.000000
SAMPLE_PACKAGE         0.000000
dtype: float64


Passons maintenant au second tableau contenant 173887 lignes et 8 attributs. On remarque tout d'abord que celui-ci possède plus de lignes que le tableau sur les produits ce qui signifie qu'un produit peut être présent dans plusieurs paquets. De plus, certains attributs sont aussi présents dans le tableau sur les produits. Par conséquent, on ne les traitera pas car nous aurons les mêmes observations :

- ``NDCPACKAGECODE`` : Cette colonne est la concaténation de la colonne ``PRODUCTNDC`` et du code de chaque paquet. Elle ne doit pas contenir de valeur NULL et doit être de type string. Cependant, celle-ci contient un léger pourcentage de valeur NULL (1.349152 %). On pourra vérifier la cohérence inter-attribut avec la colonne ``PRODUCTNDC``. 

- ``PACKAGEDESCRIPTION`` : Cette colonne contient la description du paquet. Elle ne doit pas contenir de valeur NULL et doit être de type string. Elle contient aussi le ``NDCPACKAGECODE``, on pourra donc vérifier la cohérence inter-attributs avec cette dite colonne.

- ``SAMPLE_PACKAGE`` : Cette colonne permet de savoir si le paquet est distribué en tant qu'échantillon ou pas. Les valeurs possibles sont 'Y' ou 'N' et c'est une règle à vérifier.

### Question 3

Dans cette question, nous allons résoudre les incohérences posées par le non respect des règles décrites dans la question 1 & question 2. On ne traitera pas les problèmes posés par les valeurs manquantes vu que cela sera fait à la prochaine question. Par ailleurs, par résoudre, on veut dire supprimer dans la majorité des cas (lorsque le nombre d'incohérence est très élevé et qu'il n'est pas possible de faire au cas par cas).

#### Table produit
* ##### PRODUCTID & PRODUCTNDC

In [463]:
# fix inter-incoherence between col1 and col2 by checking if col2 is a substring or not of col1
def fixInterIncoherence(df, col1, col2):
    # dont treat rows with nan values
    dfFiltered = df[df.loc[:,[col1,col2]].notnull().all(1)].loc[:,[col1,col2]]
    col1 = dfFiltered.iloc[:,0]
    col2 = dfFiltered.iloc[:,1]

    # get idx where col2 is a not substring of col1
    idxToDrop = dfFiltered.index[[x[0] not in x[1] for x in zip(col2, col1)]]

    # drop rows
    df.drop(index=idxToDrop, inplace=True)
    print("Number of incoherences dropped:", len(idxToDrop))

# fix the inter-incoherence between the productID and the productNDC
fixInterIncoherence(dfProd, 'PRODUCTID', 'PRODUCTNDC')

Number of incoherences dropped: 513


On voit qu'il y a 513 lignes qui ne respectent pas la règle de cohérence entre les colonnes ``PRODUCTID`` et ``PRODUCTNDC``.

* ##### PRODUCTTYPENAME

In [464]:
dfProd['PRODUCTTYPENAME'].unique()

array(['HUMAN OTC DRUG', 'HUMAN PRESCRIPTION DRUG', 'VACCINE',
       'PLASMA DERIVATIVE', 'CELLULAR THERAPY',
       'NON-STANDARDIZED ALLERGENIC', 'STANDARDIZED ALLERGENIC'],
      dtype=object)

On remarque que les données contenues dans la colonne ``PRODUCTTYPENAME`` n'ont rien d'aberrant : il n'y a rien à modifier pour cette colonne donc.

* ##### DOSAGEFORMNAME & ROUTENAME & MARKETINGCATEGORYNAME

In [465]:
# load fdc txt file as a set
def loadDataFromFDA(file):
    with open(file) as f:
        return [line.rstrip('\n').upper() for line in f]
    
# check the coherence between the fdc data and the col in df
# the split option is used when the col contains a list of element
def checkDataFromFDA(df, col, fda, split=False):
    # get the corresponding col without nan values
    dfFiltered = df[df[col].notnull()][col]
    
    # idx to be dropped
    idxToDrop = list()

    # if split is true, then split using the ';' delimiter
    if split:
        dfFiltered = dfFiltered.str.split(';')
        idxToDrop = dfFiltered.index[[not all(y.strip().upper() in fda for y in x) for x in dfFiltered]]

    else:
        # check for each element in dfFiltered if they are in fda and get the corresponding indexes
        idxToDrop = dfFiltered.index[[not x.strip().upper() in fda for x in dfFiltered]]

    # drop rows
    df.drop(index=idxToDrop, inplace=True)
    print("Number of incoherences dropped:", len(idxToDrop))

# dosage   
dosageFDA = loadDataFromFDA('./DosageForms.txt')
checkDataFromFDA(dfProd, 'DOSAGEFORMNAME', dosageFDA)

# route 
routeFDA = loadDataFromFDA('./RouteNames.txt')
checkDataFromFDA(dfProd, 'ROUTENAME', routeFDA, True)

# cat
MarketingCatFDA = loadDataFromFDA('./MarketingCatNames.txt')
checkDataFromFDA(dfProd, 'MARKETINGCATEGORYNAME', MarketingCatFDA)

Number of incoherences dropped: 0
Number of incoherences dropped: 0
Number of incoherences dropped: 0


On remarque que les colonnes ``DOSAGEFORMNAME``, ``ROUTENAME`` et ``MARKETINGCATEGORYNAME`` n'ont aucune incohérence et contiennent bien les données de la FDA.

* ##### STARTMARKETINGDATE & ENDMARKETINGDATE & LISTING_RECORD_CERTIFIED_THROUGH

In [466]:
# convert col in the given df into a date column
def convertDate(df, col):
    # convert into date while setting invalid parsing to NaT
    df[col] = pd.to_datetime(df[col], format="%Y%m%d", errors='coerce')
    
# convert into dates
convertDate(dfProd, 'STARTMARKETINGDATE')
convertDate(dfProd, 'ENDMARKETINGDATE')
convertDate(dfProd, 'LISTING_RECORD_CERTIFIED_THROUGH')

Maintenant que nous avons transformé les colonnes en date lisible, on doit vérifier si la date de début est inférieure à celle de fin. De même pour la date d'expiration de l'inscription du produit qui doit être comparée à celle d'aujourd'hui.

In [467]:
# check if d1 < d2 in the given df, and if it is the case, the function drops these rows
# if today is equal to True then the function compare : current date < d1
def compareDate(df, d1, d2, today=False):
    # compare to get a mask
    mask = df[d1] < df[d2] if not today else datetime.datetime.now() < df[d1]
    # drop rows
    df.drop(index=df[mask].index, inplace=True)
    print("Number of incoherences dropped:", mask.sum())

# check if start < end
compareDate(dfProd, 'ENDMARKETINGDATE', 'STARTMARKETINGDATE')
# check if the expiration end is coherent with the current date
compareDate(dfProd, 'LISTING_RECORD_CERTIFIED_THROUGH', _, True)

Number of incoherences dropped: 0
Number of incoherences dropped: 0


Il n'y a aucun problème d'incohérence inter-attributs. Il faut maintenant vérifier si nous n'avons pas des dates aberrantes :

In [468]:
# print min and max date for the given col in df
def printMinMaxDate(df, col):
    print("Column : {}, Max date: {}, min date: {}".format(col, df[col].min(), df[col].max()))

# print
printMinMaxDate(dfProd, 'STARTMARKETINGDATE')
printMinMaxDate(dfProd, 'ENDMARKETINGDATE')
printMinMaxDate(dfProd, 'LISTING_RECORD_CERTIFIED_THROUGH')

Column : STARTMARKETINGDATE, Max date: 1900-01-01 00:00:00, min date: 2020-02-14 00:00:00
Column : ENDMARKETINGDATE, Max date: 2020-02-15 00:00:00, min date: 2039-08-31 00:00:00
Column : LISTING_RECORD_CERTIFIED_THROUGH, Max date: 2020-12-31 00:00:00, min date: 2021-12-31 00:00:00


La FDA a été fondée en 1906, il semblerait donc que la date 1900-01-01 soit aberrante. On peut donc la supprimer :

In [469]:
# get mask
mask = dfProd['STARTMARKETINGDATE'] == dfProd['STARTMARKETINGDATE'].min()
# drop rows
dfProd.drop(index=dfProd[mask].index, inplace=True)
# recheck
printMinMaxDate(dfProd, 'STARTMARKETINGDATE')

Column : STARTMARKETINGDATE, Max date: 1900-01-07 00:00:00, min date: 2020-02-14 00:00:00


Nous obtenons encore une date aberrante, il faut donc la supprimer :

In [470]:
# get mask
mask = dfProd['STARTMARKETINGDATE'] == dfProd['STARTMARKETINGDATE'].min()
# drop rows
dfProd.drop(index=dfProd[mask].index, inplace=True)
# recheck
printMinMaxDate(dfProd, 'STARTMARKETINGDATE')

Column : STARTMARKETINGDATE, Max date: 1907-04-15 00:00:00, min date: 2020-02-14 00:00:00


* ##### APPLICATIONNUMBER

In [471]:
# check the inter-incoherence between the app number and the marketing cat
def checkAppNumber(df):
    # dont treat rows with nan values
    dfFiltered = dfProd[dfProd.iloc[:,[10,11]].notnull().all(1)].iloc[:,[10,11]]
    col1 = dfFiltered.iloc[:,0]
    col2 = dfFiltered.iloc[:,1]

    # get idx where the inter-incoherence is not respected
    idxToDrop = list()
    for x in zip(col1, col2, col1.index):
        # get category
        cat = x[0].split(' ')[0]

        # check if the application number contains a single digit
        if not any(i.isdigit() for i in x[1]):
            idxToDrop.append(x[2])
        
        # check if cat is a substring of the application number
        elif cat not in x[1]:
            if cat == 'OTC':
                if 'part' not in x[1]:
                    idxToDrop.append(x[2])
            elif cat == 'NDA':
                if 'BN' not in x[1]:
                    idxToDrop.append(x[2])
            elif cat == 'ANDA':
                if 'BA' not in x[1]:
                    idxToDrop.append(x[2])
            else:
                idxToDrop.append(x[2])
                
    # print
    print("Number of incoherences:", len(idxToDrop))
    display(df.loc[idxToDrop, df.columns[[10,11]]])
        
checkAppNumber(dfProd)

Number of incoherences: 2


,MARKETINGCATEGORYNAME,APPLICATIONNUMBER
26428,OTC MONOGRAPH FINAL,333D
90014,OTC MONOGRAPH NOT FINAL,part


On remarque que nous avons que deux incohérences, nous pouvons donc les corriger : 
- pour la première, il suffit d'ajouter le préfixe ``part`` qui semble être oublié.
- concernant la seconde, c'est le numéro qui semble être oublié, ce qui nous oblige à supprimer cette ligne.

In [472]:
# add part prefix
dfProd.loc[26428, 'APPLICATIONNUMBER'] = 'part333D'
# drop the row missing the application number
dfProd.drop(index=90014, inplace=True)
# recheck
checkAppNumber(dfProd)

Number of incoherences: 0


,MARKETINGCATEGORYNAME,APPLICATIONNUMBER


* ##### SUBSTANCENAME & ACTIVE_NUMERATOR_STRENGTH & ACTIVE_INGRED_UNIT

In [473]:
# check the inter-incoherence between the substance name, the active strength and the active ingredient
def checkSubstance(df):
    # dont treat rows with nan values
    dfFiltered = dfProd[dfProd.iloc[:,[13,14,15]].notnull().all(1)].iloc[:,[13,14,15]]
    col1 = dfFiltered.iloc[:,0]
    col2 = dfFiltered.iloc[:,1]
    col3 = dfFiltered.iloc[:,2]

    # get idx where the inter-incoherence is not respected
    idxToDrop = list()
    for x in zip(col1, col2, col3, col1.index):
        # split to list
        l1 = x[0].split(';')
        l2 = x[1].split(';')
        l3 = x[2].split(';')

        # check size
        if not len(l1) == len(l2) == len(l3):
            idxToDrop.append(x[3])
            pass

        # check digits
        for d in l2:
            try:
                float(d)
            except:
                idxToDrop.append(x[3])
                break

    # print
    print("Number of incoherences:", len(idxToDrop))
    display(df.loc[idxToDrop, df.columns[[13,14,15]]])

checkSubstance(dfProd)

Number of incoherences: 1


,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT
90536,GLYCERIN; HYDROLYZED SOY PROTEIN (ENZYMATIC; 2...,10; .12,g/100g; g/100g


On remarque que nous avons qu'une incohérence et elle est causée par l'ajout d'un point virgule au niveau du ``SUBSTANCENAME`` :

In [474]:
# print
dfProd.loc[90536,'SUBSTANCENAME']

'GLYCERIN; HYDROLYZED SOY PROTEIN (ENZYMATIC; 2000 MW)'

Il suffit juste de supprimer le point virgule pour fixer le problème :

In [475]:
# fix
dfProd.loc[90536,'SUBSTANCENAME'] = 'GLYCERIN; HYDROLYZED SOY PROTEIN (ENZYMATIC 2000 MW)'
# recheck
checkSubstance(dfProd)

Number of incoherences: 0


,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT


* ##### DEASCHEDULE & NDC_EXCLUDE_FLAG

In [476]:
# check dea schedule
dfProd.loc[:,'DEASCHEDULE'].unique()

array([nan, 'CV', 'CIV', 'CIII', 'CII'], dtype=object)

La colonne contient aucune valeur aberrante.

In [477]:
# check ndc flag
dfProd.loc[:,'NDC_EXCLUDE_FLAG'].unique()

array(['N'], dtype=object)

La colonne ne contient qu'une valeur qu'est ``N``, ce qui était attendu.

#### Table paquet

* ##### PRODUCTID & PRODUCTNDC

In [478]:
# fix the inter-incoherence between the productID and the productNDC    
fixInterIncoherence(dfPack, 'PRODUCTID', 'PRODUCTNDC')

Number of incoherences dropped: 519


On voit qu'il y a 519 lignes qui ne respectent pas la règle de cohérence entre les colonnes ``PRODUCTID`` et ``PRODUCTNDC``.

* ##### NDCPACKAGECODE

In [479]:
# fix the inter-incoherence between the ndc package code and the product NDC
fixInterIncoherence(dfPack, 'NDCPACKAGECODE', 'PRODUCTNDC')

Number of incoherences dropped: 155


On voit qu'il y a 155 lignes qui ne respectent pas la règle de cohérence entre les colonnes ``PRODUCTNDC`` et ``NDCPACKAGECODE``.

* ##### PACKAGEDESCRIPTION

In [480]:
# fix the inter-incoherence between the package description and the product NDC
fixInterIncoherence(dfPack, 'PACKAGEDESCRIPTION', 'NDCPACKAGECODE')

Number of incoherences dropped: 0


On voit qu'il y a 0 ligne qui ne respectent pas la règle de cohérence entre les colonnes ``PACKAGEDESCRIPTION`` et ``NDCPACKAGECODE``.

* ##### STARTMARKETINGDATE & ENDMARKETINGDATE

In [481]:
# convert into dates
convertDate(dfPack, 'STARTMARKETINGDATE')
convertDate(dfPack, 'ENDMARKETINGDATE')

# check if start < end
compareDate(dfProd, 'ENDMARKETINGDATE', 'STARTMARKETINGDATE')

Number of incoherences dropped: 0


Il n'y a aucun problème d'incohérence inter-attributs. Il faut maintenant vérifier si nous n'avons pas des dates aberrantes :

In [482]:
printMinMaxDate(dfPack, 'STARTMARKETINGDATE')
printMinMaxDate(dfPack, 'ENDMARKETINGDATE')

Column : STARTMARKETINGDATE, Max date: 1900-01-01 00:00:00, min date: 2020-02-14 00:00:00
Column : ENDMARKETINGDATE, Max date: 2020-02-15 00:00:00, min date: 2039-08-31 00:00:00


On retombe sur le même problème que pour la table produit, il suffit juste de le fixer aussi de la même manière :

In [483]:
# get mask
mask = dfPack['STARTMARKETINGDATE'] == dfPack['STARTMARKETINGDATE'].min()
# drop rows
dfPack.drop(index=dfPack[mask].index, inplace=True)
# recheck
printMinMaxDate(dfPack, 'STARTMARKETINGDATE')

Column : STARTMARKETINGDATE, Max date: 1900-01-07 00:00:00, min date: 2020-02-14 00:00:00


In [484]:
# get mask
mask = dfProd['STARTMARKETINGDATE'] == dfProd['STARTMARKETINGDATE'].min()
# drop rows
dfProd.drop(index=dfProd[mask].index, inplace=True)
# recheck
printMinMaxDate(dfProd, 'STARTMARKETINGDATE')

Column : STARTMARKETINGDATE, Max date: 1919-01-01 00:00:00, min date: 2020-02-14 00:00:00


* ##### NDC_EXCLUDE_FLAG & SAMPLE_PACKAGE

In [485]:
# check ndc flag
dfPack.loc[:,'NDC_EXCLUDE_FLAG'].unique()

array(['N'], dtype=object)

In [486]:
# check sample package
dfPack.loc[:,'SAMPLE_PACKAGE'].unique()

array(['N', 'Y'], dtype=object)

Les deux colonnes contiennent uniquement des bonnes données, il n'y a rien à modifier donc.

### Question 4

Comme nous avons pu le voir auparavant, les deux tableaux possèdent des valeurs NULL. Cependant, l'importance accordée à la présence d'une valeur manquante dans une colonne diffère de l'une de l'autre. En effet, cela va dépendre de la quantité de valeurs NULL et si la valeur NULL encode une information. En effet :

- ``PROPRIETARYNAME`` : Cette colonne contient contient un faible niveau de valeur NULL (0.006435 %), on peut donc drop les valeurs manquantes.

- ``PROPRIETARYNAMESUFFIX`` : Cette colonne peut contenir des valeurs NULL et une valeur manquante encode une optionalité d'où un niveau élevé de valeur NULL (89 %). On peut donc l'ignorer. 

- ``NONPROPRIETARYNAME`` : Cette colonne contient contient un faible niveau de valeur NULL (0.004290 %), on peut donc drop les valeurs manquantes.

- ``ROUTENAME`` : Cette colonne contient contient un faible niveau de valeur NULL (2.072117 %), on peut donc drop les valeurs manquantes.

- ``ENDMARKETINGDATE`` : Cette colonne peut contenir des valeurs NULL et une valeur manquante encode de l'information (i.e pas de date de fin) d'où un niveau élevé de valeur NULL (95 %). On peut donc l'ignorer.

- ``APPLICATIONNUMBER`` : Cette colonne peut contenir des valeurs NULL et une valeur manquante encode de l'information (i.e le produit n'est pas approuvé par la FDA). On peut donc remplacer les valeurs NULL par 'Unapproved drugs'.

- ``SUBSTANCENAME``, ``ACTIVE_NUMERATOR_STRENGTH``, ``ACTIVE_INGRED_UNIT`` : Ces 3 attributs sont liés et contiennent un faible niveau identique de valeurs NULL (2.476458 %). On peut donc drop les valeurs manquantes.

- ``PHARM_CLASSES`` : Voir les prochaines questions (donc à ignorer).

- ``DEASCHEDULE`` : Cette colonne peut contenir des valeurs NULL et une valeur manquante encode de l'information (i.e le pouvoir de dépendance du produit est inexistant). On peut donc remplacer les valeurs NULL par 'C0'.  

- ``LISTING_RECORD_CERTIFIED_THROUGH`` : Cette colonne contient contient un faible niveau de valeur NULL (4.638667 %), on peut donc drop les valeurs manquantes.

Finalement, ici, nous avons uniquement pris en compte les colonnes pour lesquels nous n'avons aucun moyen de récupérer les valeurs NULL d'où la mise en place d'une suppression.

In [487]:
# drop NaN
dfProd.dropna(subset=['PROPRIETARYNAME','NONPROPRIETARYNAME', 
                      'ROUTENAME', 'SUBSTANCENAME','ACTIVE_NUMERATOR_STRENGTH',
                      'ACTIVE_INGRED_UNIT','LISTING_RECORD_CERTIFIED_THROUGH'], inplace=True)

# replace NaN
dfProd['APPLICATIONNUMBER'].fillna('Unapproved drugs', inplace=True)
dfProd['DEASCHEDULE'].fillna('C0', inplace=True)

Dans cette section, nous allons essayer de mettre en place un moyen de récupérer des valeurs manquantes dans : 

- Table produit : ``PRODUCTID``
- Table packet : ``PRODUCTNDC`` et ``NDCPACKAGECODE``

L'idée est d'utiliser la colonne ``PACKAGEDESCRIPTION`` (qui ne possède aucune valeur manquante) pour récupérer toutes les valeurs manquantes de la colonne ``NDCPACKAGECODE``. Par la suite, on utilise la même astuce pour récupérer ceux de la colonne ``PRODUCTNDC`` à l'aide de ``NDCPACKAGECODE``.

Maintenant, pour obtenir les valeurs manquantes de ``PRODUCTID``, on va comparer la colonne ``PRODUCTNDC`` de la table produit (qui ne possède pas de valeurs NULL) à celle de la table packet : de cette manière, on peut recopier les valeurs qui matchent depuis la colonne ``PRODUCTID`` de la table packet vers celle des produits.

In [488]:
# get idx of nan values in ndc package code
idx = dfPack.loc[pd.isna(dfPack["NDCPACKAGECODE"]), :].index

# list to cache result
NDCPackCode = list()
NDCProdCode = list()

# for each nan value in ndc package code 
for i in idx:
    # use regex to get the ndc package code from the package description
    code = re.findall(r"\(([^)]+)\)", dfPack["PACKAGEDESCRIPTION"][i])[0]
    # save in ndc package code column
    NDCPackCode.append(code)

# replace
dfPack.loc[idx, "NDCPACKAGECODE"] = NDCPackCode

# get split
split = dfPack.loc[:, "NDCPACKAGECODE"].str.split('-', n = 3, expand=True)
# replace
dfPack["PRODUCTNDC"] = split[0] + '-' + split[1]

# check
print("Percentage of missing values for each columns:")
print(dfPack.isnull().sum() * 100 / len(dfPack))

Percentage of missing values for each columns:
PRODUCTID              0.000000
PRODUCTNDC             0.000000
NDCPACKAGECODE         0.000000
PACKAGEDESCRIPTION     0.000000
STARTMARKETINGDATE     0.140311
ENDMARKETINGDATE      96.364046
NDC_EXCLUDE_FLAG       0.000000
SAMPLE_PACKAGE         0.000000
dtype: float64


Nous n'avons plus de valeurs NULL, vérifions tout de même la cohérence de cette stratégie :

In [489]:
# fix the inter-incoherence between the productID and the productNDC    
fixInterIncoherence(dfPack, 'PRODUCTID', 'PRODUCTNDC')

# fix the inter-incoherence between the ndc package code and the product NDC
fixInterIncoherence(dfPack, 'NDCPACKAGECODE', 'PRODUCTNDC')

# fix the inter-incoherence between the package description and the product NDC
fixInterIncoherence(dfPack, 'PACKAGEDESCRIPTION', 'NDCPACKAGECODE')

Number of incoherences dropped: 0
Number of incoherences dropped: 0
Number of incoherences dropped: 0


Aucune incohérence n'a été trouvée, on peut donc maintenant passer à la table produit :

In [490]:
# get idx of nan values in product id
idx = dfProd.loc[pd.isna(dfProd["PRODUCTID"]), :].index

# get the corresponding ndc product code
NDCProdCode = dfProd.loc[idx, "PRODUCTNDC"]

# get the corresponding product id from the package table
prodID = dfPack.query('PRODUCTNDC in @NDCProdCode')['PRODUCTID'].drop_duplicates()

# replace
dfProd.loc[idx,'PRODUCTID'] = prodID.values

# check
print("Percentage of missing values for each columns:")
print(dfProd.isnull().sum() * 100 / len(dfProd))

Percentage of missing values for each columns:
PRODUCTID                             1.787684
PRODUCTNDC                            0.000000
PRODUCTTYPENAME                       0.000000
PROPRIETARYNAME                       0.000000
PROPRIETARYNAMESUFFIX                89.705729
NONPROPRIETARYNAME                    0.000000
DOSAGEFORMNAME                        0.000000
ROUTENAME                             0.000000
STARTMARKETINGDATE                    0.000000
ENDMARKETINGDATE                    100.000000
MARKETINGCATEGORYNAME                 0.000000
APPLICATIONNUMBER                     0.000000
LABELERNAME                           0.000000
SUBSTANCENAME                         0.000000
ACTIVE_NUMERATOR_STRENGTH             0.000000
ACTIVE_INGRED_UNIT                    0.000000
PHARM_CLASSES                        52.585757
DEASCHEDULE                           0.000000
NDC_EXCLUDE_FLAG                      0.000000
LISTING_RECORD_CERTIFIED_THROUGH      0.000000
dtype: float6

In [491]:
# fix the inter-incoherence between the productID and the productNDC    
fixInterIncoherence(dfProd, 'PRODUCTID', 'PRODUCTNDC')

Number of incoherences dropped: 0


Nous n'avons plus de valeurs NULL et la cohérence est toujours respectée. Par ailleurs, on remarque que la colonne ``ENDMARKETINGDATE`` contient que des valeurs NULL suite au nettoyage : on peut donc la supprimer au niveau des deux tableaux car elle n'apporte que très peu d'information.

In [492]:
# remove ENDMARKETINGDATE column
dfProd.drop('ENDMARKETINGDATE', axis=1, inplace=True)
dfPack.drop('ENDMARKETINGDATE', axis=1, inplace=True)

### Question 5

Pour supprimer les doublons, nous allons partir sur le fait que deux objets sont parfaitement identiques si et seulement si ils ont les mêmes attributs à l'exception de ``PRODUCTID``.

In [512]:
# delete duplicates
def deleteDuplicates(df):
    # get columns except the product id
    col = df.columns[1:len(df.columns)]
    # print the number of duplicated rows 
    print('Number of duplicated rows:', df.duplicated(col).sum())
    # delete duplicates
    df.drop_duplicates(subset=col, inplace=True)

# product df
print("Product table:")
deleteDuplicates(dfProd)

# package df
print("Package table:")
deleteDuplicates(dfPack)

Product table:
Number of duplicated rows: 592
Package table:
Number of duplicated rows: 1


Regardons maintenant le taux de lignes supprimées suite à ce nettoyage :

In [523]:
print('Percentage of deleted rows (product table):', (93238 - dfProd.shape[0]) / 93238 * 100)
print('Percentage of deleted rows (product table):', (173887 - dfPack.shape[0]) / 173887 * 100)

Percentage of deleted rows (product table): 8.242347540702289
Percentage of deleted rows (product table): 0.40313536952158585


Finalement, nous n'avons perdu que très peu de données suite à toutes ces questions de nettoyage.

### Question 6

In [525]:
# TODO